In [1]:
# Setup paths for trained model etc
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'C:/Users/Sreesha/Documents/Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

In [2]:
labels = [{'name':'1', 'id':1},
          {'name':'2', 'id':2},
          {'name':'3', 'id':3},
          {'name':'4', 'id':4},
          {'name':'5', 'id':5},
          {'name':'6', 'id':6},
          {'name':'7', 'id':7},
          {'name':'8', 'id':8},
          {'name':'9', 'id':9},
          {'name':'a', 'id':10},
          {'name':'b', 'id':11},
          {'name':'c', 'id':12},
          {'name':'d', 'id':13},
          {'name':'e', 'id':14},
          {'name':'f', 'id':15},
          {'name':'g', 'id':16},
          {'name':'h', 'id':17},
          {'name':'i', 'id':18},
          {'name':'j', 'id':19},
          {'name':'k', 'id':20},
          {'name':'l', 'id':21},
          {'name':'m', 'id':22},
          {'name':'n', 'id':23},
          {'name':'o', 'id':24},
          {'name':'p', 'id':25},
          {'name':'q', 'id':26},
          {'name':'r', 'id':27},
          {'name':'s', 'id':28},
          {'name':'t', 'id':29},
          {'name':'u', 'id':30},
          {'name':'v', 'id':31},
          {'name':'w', 'id':32},
          {'name':'x', 'id':33},
          {'name':'y', 'id':34},
          {'name':'z', 'id':35}]

In [3]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [4]:
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [5]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [6]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [7]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-21')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [8]:
import numpy as np

In [9]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [10]:
acc_of_each_sign = {'1':0,
          '2':0,
          '3':0,
          '4':3,
          '5':0,
          '6':0,
          '7':0,
          '8':0,
          '9':0,
          'a':0,
          'b':0,
          'c':0,
          'd':0,
          'e':0,
          'f':0,
          'g':0,
          'h':0,
          'i':0,
          'j':0,
          'k':0,
          'l':0,
          'm':0,
          'n':0,
          'o':0,
          'p':0,
          'q':0,
          'r':0,
          's':0,
          't':0,
          'u':0,
          'v':0,
          'w':0,
          'x':0,
          'y':0,
          'z':0
         }

In [11]:
from keras.preprocessing.image import img_to_array, load_img
paths = ['C:/Users/Sreesha/Documents/Capstone project - Sign language/0Indian sign language trial 3/Tensorflow/workspace/images/test']
actual_count = 0
detected_count = 0
for path in paths:
    for root, directories, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith(".jpg"):
                fullpath = os.path.join(root, filename)
                actual = filename[0] 
                frame = load_img(fullpath)
                image_np = np.array(frame)
                input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
                detections = detect_fn(input_tensor)
    
                num_detections = int(detections.pop('num_detections'))
                detections = {key: value[0, :num_detections].numpy()
                              for key, value in detections.items()}
                detections['num_detections'] = num_detections

                # detection_classes should be ints.
                detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
                label_id_offset = 1
                image_np_with_detections = image_np.copy()
                detected_sign = (category_index[detections['detection_classes'][0]+label_id_offset]).get('name')
                actual_count += 1
                if actual == detected_sign:
                    detected_count +=1
                    acc_of_each_sign[actual] += 1
                    
                
print((detected_count*100) / actual_count)

for i in acc_of_each_sign.keys():
    print(i,acc_of_each_sign[i]*10)

92.57142857142857
1 100
2 100
3 100
4 30
5 100
6 90
7 100
8 100
9 100
a 100
b 100
c 100
d 100
e 100
f 100
g 80
h 100
i 70
j 100
k 100
l 60
m 100
n 80
o 100
p 100
q 90
r 90
s 100
t 100
u 80
v 100
w 100
x 100
y 100
z 100
